In [7]:
import cv2
import numpy as np

def upscale_image(image_path, scale=2):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    upscaled = cv2.resize(image, (width * scale, height * scale), interpolation=cv2.INTER_CUBIC)
    return upscaled

def unsharp_mask(image, sigma=1.0, strength=1.5):
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    
    # Sharpen the image
    sharpened = cv2.addWeighted(image, 1.0 + strength, blurred, -strength, 0)
    
    return sharpened

# Example usage
image_path = "Screenshot from 2025-04-02 00-48-50.png"

# Upscale the image
upscaled = upscale_image(image_path)

# Remove Gaussian Blur
deblurred = unsharp_mask(upscaled)

# Save the result
cv2.imwrite("output.jpg", deblurred)


True

In [5]:
pip install opencv-contrib-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import cv2
import numpy as np
from scipy.signal import wiener

def gaussian_kernel(size, sigma):
    k = cv2.getGaussianKernel(size, sigma)
    kernel = np.outer(k, k)
    return kernel

def wiener_deconvolution(blurred, kernel, noise_power=0.000005):
    kernel /= np.sum(kernel)
    
    img_fft = np.fft.fft2(blurred)
    kernel_fft = np.fft.fft2(kernel, s=blurred.shape)
    
    kernel_fft_conj = np.conj(kernel_fft)
    denominator = kernel_fft * kernel_fft_conj + noise_power
    result_fft = (img_fft * kernel_fft_conj) / denominator
    
    result = np.fft.ifft2(result_fft).real
    return np.clip(result, 0, 255).astype(np.uint8)

def deblur_image(path, kernel_size, sigma):
    blurred = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    
    kernel = gaussian_kernel(kernel_size, sigma)
    deblurred = wiener_deconvolution(blurred, kernel)

    cv2.imwrite("deblurred.png", deblurred)

deblur_image("txyx.png", 25, 7)